In [1]:
#nativos
import os
import time
import unicodedata
import gc

#terceros
import pandas as pd
import numpy as np
import pymssql
import missingno as msnolistadconoce
from collections import Counter

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

In [2]:
from utils_alertas import *
try:
    connection_market.close()
except:
    pass

#conneciom MARKET
connection_market = pymssql.connect(**dicc_conection_market)

EJEMPLO:
202104 mes cerrado de la cosecha de 202104 [202011, 202012, 202101, 202102, 202103, 202104] [202102, 202103, 202104] 
 historia 
 [202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104] 
 ---------------------------------------------------------------------------------------------------- 
 [202105, 202106] 
 ---------------------------------------------------------------------------------------------------- 
 [202005, 202006]
EJEMPLO:
202003 mes cerrado de la cosecha de 202003 [201910, 201911, 201912, 202001, 202002, 202003] [202001, 202002, 202003] 
 historia 
 [201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001, 202002, 202003] 
 ---------------------------------------------------------------------------------------------------- 
 [202004, 202005] 
 ---------------------------------------------------------------------------------------------------- 
 [201904, 201905]


### Clientes Universo

In [3]:
sql_clientes = """
    SELECT * FROM ODS.HD_VMVP_BASE_ALERTA_CARTERA_BE WITH (NOLOCK)
"""

clientes = pd.read_sql(
   sql_clientes , connection_market
)
clientes.head(3)

,FEVE,PERIODO,CODUNICOCLI,NOMBRE_COMPLETO,FLG_VIGENTE,FLG_SUSPENDIDO,FLG_VENCIDA,FLG_CANCELADO,FLG_MONOLINEA,FLG_SOLO_REACTIVA,...,CLASIF_SBS,CLASIF_IBK,DEPARTAMENTO,FEC_INGRESO,FEC_PRIMER_PRODUCTO_APERTURADO,SEI_MCDO,TIPDOC,CODDOC,NUM_RUC,SEGMENTO_CLIENTE
0,GARANTIZAR,202008,0003041431,CERAMICOS PERUANOS,1,0,1,0,0,0,...,NORMAL,NORMAL,LIMA,1995-02-03,1999-05-07,CONSTRUCCIÓN,2,20100132916,20100132916,S2
1,EX-FEVE,202008,0003142478,BIOCON DEL PERU,1,1,0,0,0,0,...,NORMAL,NORMAL,CALLAO,1995-02-24,2009-10-23,INDUSTRIA,2,20100013747,20100013747,S1
2,EX-FEVE,202008,0003107943,ARENERA JAEN,0,1,1,1,0,0,...,NORMAL,NORMAL,LA LIBERTAD,1995-02-17,1995-02-17,TRANSPORTE,2,20114022838,20114022838,S1


In [4]:
clientes.shape, clientes.columns

((26914, 39),
 Index(['FEVE', 'PERIODO', 'CODUNICOCLI', 'NOMBRE_COMPLETO', 'FLG_VIGENTE',
        'FLG_SUSPENDIDO', 'FLG_VENCIDA', 'FLG_CANCELADO', 'FLG_MONOLINEA',
        'FLG_SOLO_REACTIVA', 'FLG_POSICION', 'FUNNEL_1', 'FUNNEL_2', 'FUNNEL_3',
        'RIESGO', 'COD_GRUPO_ECONOMICO', 'DES_GRUPO_ECONOMICO', 'ZONAL',
        'ENCARGADO', 'NOMBRE_JEFE', 'CLASIFICACION_LIST_360', 'FUNNEL_4',
        'FUNNEL_ALERTA', 'DESPRIORIZACION_ADM', 'RIESGO_DIRECTO',
        'RIESGO_INDIRECTO', 'RIESGO_REACTIVA_NO_C', 'BUCKET', 'ES_ACTIVO',
        'CLASIF_SBS', 'CLASIF_IBK', 'DEPARTAMENTO', 'FEC_INGRESO',
        'FEC_PRIMER_PRODUCTO_APERTURADO', 'SEI_MCDO', 'TIPDOC', 'CODDOC',
        'NUM_RUC', 'SEGMENTO_CLIENTE'],
       dtype='object'))

In [5]:
print(clientes.shape)
clientes = clientes[
    (clientes['ZONAL'].notnull()) & (clientes['ZONAL'] != '')
]
print(clientes.shape)

(26914, 39)
(26914, 39)


In [6]:
pd.crosstab(
    clientes['ZONAL'].fillna('-'), clientes['PERIODO']
)

PERIODO,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105
ZONAL,,,,,,,,,,
BC GRUPO 1,0,0,0,0,0,185,186,190,190,189
BC GRUPO 2,0,0,0,0,0,217,216,226,225,227
BEL ZONAL 1,275,277,275,278,279,275,276,281,278,276
BEL ZONAL 2,1216,1244,1235,1229,1243,1246,1248,1232,1226,1224
BEL ZONAL 3,209,211,212,209,209,207,210,210,211,209
BEP ZONAL 1,282,294,300,297,298,296,298,290,291,297
BEP ZONAL 2,236,249,255,248,246,241,243,242,237,239
BEP ZONAL 3,210,224,230,233,230,225,223,232,232,235


### Inclusión de FEVE RE-AJUSTADO

In [7]:
sql_ = """
    SELECT * FROM ODS.HM_FEVE_REAJUSTADO_SEGUIR_COVID WITH (NOLOCK)
"""
reajustado = pd.read_sql(
   sql_ , connection_market
)
reajustado['CODUNICOCLI'] = reajustado['CU'].copy()
reajustado.head(3)

,CU,CLIENTE,SEGUIR COVID,PERIODO,SEGUIR_COVID,CODUNICOCLI
0,0012430609,SOCIEDAD AGRICOLA RAPEL,SI,202007,1,0012430609
1,0009230888,MIXERCON,SI,202007,1,0009230888
2,0013693786,MOVIL BUS,SI,202007,1,0013693786


In [8]:
reajustado['PERIODO'].value_counts().sort_index(ascending=False)

202105     65
202104    240
202103    232
202102    240
202101    239
202012    246
202011    260
202010    263
202009    269
202008     71
202007     44
Name: PERIODO, dtype: int64

In [9]:
clientes['PERIODO'].value_counts().sort_index(ascending=False)

202105    2896
202104    2890
202103    2903
202102    2900
202101    2892
202012    2505
202011    2494
202010    2507
202009    2499
202008    2428
Name: PERIODO, dtype: int64

In [10]:
reajustado.dtypes

CU              object
CLIENTE         object
SEGUIR COVID    object
PERIODO         object
SEGUIR_COVID     int64
CODUNICOCLI     object
dtype: object

In [11]:
print(clientes.shape)
clientes = clientes.merge(
    reajustado[['CODUNICOCLI', 'PERIODO', 'SEGUIR_COVID']], how='left', on=['CODUNICOCLI', 'PERIODO']
)
clientes.head()
print(clientes.shape)

(26914, 39)
(26914, 40)


In [12]:
pd.crosstab(
    clientes['FEVE'], clientes['PERIODO']
)

PERIODO,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105
FEVE,,,,,,,,,,
EX-FEVE,622,617,619,632,640,813,817,828,847,856
GARANTIZAR,317,253,189,160,142,124,124,102,91,88
RECUP.,0,0,0,0,4,0,0,0,0,0
REDUCIR,72,81,93,92,93,118,117,115,107,107
SALIR,23,25,26,24,27,34,32,34,44,44
SEGUIR,159,229,286,307,320,428,434,458,441,440


In [13]:
print(clientes.shape)
clientes = clientes[clientes['FEVE'] != 'RECUP.']
print(clientes.shape)

(26914, 40)
(26910, 40)


In [14]:
### Coordnación con Nersy, se actualzan con SEGUR COVID a los que no tengan FEVE DURO

def update_feve(feve, covid):
    return 'SEGUIR COVID' if covid == 1 and feve not in ['REDUCIR', 'SALIR'] else feve


clientes['FEVE'] = clientes[['FEVE', 'SEGUIR_COVID']].apply(
    lambda _: update_feve(_[0], _[1]), axis=1
)

pd.crosstab(
    clientes['FEVE'], clientes['PERIODO']
)

PERIODO,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105
FEVE,,,,,,,,,,
EX-FEVE,621,591,611,630,635,805,812,826,843,855
GARANTIZAR,294,203,155,128,116,100,102,84,74,81
REDUCIR,72,81,93,92,93,118,117,115,107,107
SALIR,23,25,26,24,27,34,32,34,44,44
SEGUIR,114,113,131,142,161,240,240,256,236,383
SEGUIR COVID,69,203,200,201,190,226,227,222,229,65


### REVISANDO LOS FUNNELS

In [ ]:
pd.crosstab(
    clientes['FUNNEL_4'].fillna(-1), clientes['PERIODO']  # 
)

In [ ]:
pd.crosstab(
    clientes['FUNNEL_ALERTA'], clientes['PERIODO']
)

In [ ]:
#clientes.drop_duplicates(subset=['CODUNICOCLI'])[['NUM_RUC', 'NOMBRE_COMPLETO', 'ZONAL', 'ES_ACTIVO', 'BUCKET']].to_excel('CLIENTES_UNIERSO_ALERTAS_CARTERA.xlsx')

In [ ]:
unicos = clientes.drop_duplicates(subset=['CODUNICOCLI'])
unicos[['NUM_RUC', 'NOMBRE_COMPLETO', 'ZONAL', 'ES_ACTIVO', 'BUCKET']].shape

In [ ]:
unicos[['NUM_RUC', 'NOMBRE_COMPLETO', 'ZONAL', 'ES_ACTIVO', 'BUCKET']].to_excel('CLIENTES_UNIERSO_ALERTAS_CARTERA.xlsx', index=False)

In [ ]:
clientes['PERIODO_ZONAL'] = clientes['PERIODO'] + '__' + clientes['ZONAL']

In [ ]:
#for periodo in clientes[clientes['PERIODO'].astype(int) >= 202101]['PERIODO'].unique():
#    display(pd.crosstab(
#        clientes[clientes['PERIODO'] == periodo]['FUNNEL_4'], clientes[clientes['PERIODO'] == periodo]['PERIODO_ZONAL']
#    ))
#    
#    display(pd.crosstab(
#        clientes[clientes['PERIODO'] == periodo]['FUNNEL_ALERTA'], clientes[clientes['PERIODO'] == periodo]['PERIODO_ZONAL']
#    ))

In [ ]:
clientes[clientes['ZONAL'] == ''].shape[0]

In [ ]:
#cruce = pd.crosstab(
#    clientes['SEI_MCDO'], clientes['FEVE']
#)
#cruce['total_referencial'] = cruce.sum(axis=1)#
#for col in cruce.columns:
#    cruce[col] =  cruce[col] / cruce['total_referencial']
    
#cruce

### >> SEI A CRITERIO EXPERTO

In [ ]:
clientes['SEI_MCDO'] = clientes['SEI_MCDO'].fillna('DESCONOCIDO')

In [ ]:
print(set(clientes['SEI_MCDO']))

In [ ]:
text_sei_mas_preocupa = 'RESTAURANTES%,%HIDROCARBUROS%,%AUTOMOTRIZ VEHÍCULOS%,%TEXTIL'
text_sei_medio_peocupa = 'INDUSTRIA%,%CONSTRUCCIÓN%,%TRANSPORTE%,%MINERÍA%,%INMOBILIARIO'
text_sei_banca = 'RESTAURANTES%,%AUTOMOTRIZ VEHÍCULOS%,%TEXTIL,%CONSTRUCCIÓN%,%TRANSPORTE, %TURISMO Y HOSPEDAJE'

clientes['sei_letal'] = clientes['SEI_MCDO'].apply(lambda _: _.upper() in text_sei_mas_preocupa).astype(int)
clientes['sei_preocupa'] = clientes['SEI_MCDO'].apply(lambda _: _.upper() in text_sei_mas_preocupa or _.upper() in text_sei_medio_peocupa).astype(int)
clientes['sei_observado_banca'] = clientes['SEI_MCDO'].apply(lambda _: _.upper() in text_sei_banca).astype(int)

In [ ]:
clientes.shape



### >> UN VISTAZO AL RIESGO

In [ ]:
clientes['BUCKET'].value_counts()

In [ ]:
sql_tc = """
    SELECT * FROM TIPO_CAMBIO WHERE FECHA = (SELECT MAX(FECHA) FROM TIPO_CAMBIO)
"""
tc = pd.read_sql(
   sql_tc , connection_market
)
tc.head(3)

In [ ]:
ultimo_tipo_cambio = tc.iloc[0]['tc_punta']
ultimo_tipo_cambio

In [ ]:
ticket_promedio = (clientes.groupby(by=['BUCKET'])['RIESGO'].agg(['median']).apply(
    lambda _: round(_*ultimo_tipo_cambio, 1))
).to_dict()['median']

In [ ]:
def format_monetary(val):
    if val < 1000000:
        return str(round(val/1000, 1)) + ' M'
    else:
        return str(round(val/1000000, 1)) + ' MM'

clientes['TICKET_PROMEDIO_EN_RIESGO'] = clientes['BUCKET'].map(ticket_promedio)
clientes['TICKET_PROMEDIO_EN_RIESGO'] = clientes[['TICKET_PROMEDIO_EN_RIESGO', 'BUCKET']].apply(lambda _: 'Bucket {} - {}'.format(
    int(_[1]), format_monetary(_[0])).replace(" ", "_"),  axis=1)

clientes['TICKET_PROMEDIO_EN_RIESGO'].value_counts()

In [ ]:
clientes['FEVE'].value_counts()

In [ ]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

now = datetime.now()

### CREACION DE BASE

In [ ]:
clientes.shape, clientes['CODUNICOCLI'].unique().shape

In [ ]:
clientes['NUM_RUC'].isnull().sum(),  clientes['CODDOC'].isnull().sum()

In [ ]:
print(clientes.shape)
clientes = clientes[clientes['NUM_RUC'].notnull()]
print(clientes.shape)

In [ ]:
clientes['COD_UNICO'] = clientes['CODUNICOCLI'].copy()
clientes['RUC'] = clientes['NUM_RUC'].copy()
clientes['UNIVERSO_MESA'] = 1

base = clientes[['PERIODO', 'COD_UNICO', 'RUC', 'FLG_VIGENTE', 'NOMBRE_COMPLETO',
       'FLG_SUSPENDIDO', 'FLG_VENCIDA', 'FLG_CANCELADO', 'FLG_MONOLINEA',
       'FLG_SOLO_REACTIVA', 'FLG_POSICION', 'FUNNEL_1', 'FUNNEL_2', 'FUNNEL_3',
       'RIESGO', 'COD_GRUPO_ECONOMICO', 'DES_GRUPO_ECONOMICO', 'ZONAL',
       'ENCARGADO', 'NOMBRE_JEFE', 'CLASIFICACION_LIST_360', 'FUNNEL_4', 'FUNNEL_ALERTA',
       'DESPRIORIZACION_ADM','BUCKET', 'FEVE', 'ES_ACTIVO',
       'CLASIF_SBS', 'CLASIF_IBK', 'DEPARTAMENTO', 'FEC_INGRESO',
       'FEC_PRIMER_PRODUCTO_APERTURADO', 'SEI_MCDO', 'sei_preocupa', 'sei_letal', 'sei_observado_banca',
       'TICKET_PROMEDIO_EN_RIESGO', 'UNIVERSO_MESA', 'RIESGO_REACTIVA_NO_C', 'RIESGO_DIRECTO', 'RIESGO_INDIRECTO'
]]

base.head()

In [ ]:
base['CLASIF_SBS'] = base['CLASIF_SBS'].fillna('NORMAL')     # si no tiene clasificacion se pone NORMAL
base['CLASIF_IBK'] = base['CLASIF_IBK'].fillna('NORMAL')     # si no tiene clasificacion se pone NORMAL

base['CLASIF_NO_NORMAL'] = base[['CLASIF_SBS', 'CLASIF_IBK']].apply(
    lambda _: (_[0] != 'NORMAL' or _[1] != 'NORMAL') and _[0] != '' and _[1] != '', 
    axis=1
).astype(int)

In [ ]:

pd.crosstab(
    base['CLASIF_NO_NORMAL'], base['PERIODO']
)

In [ ]:
#base[base['CLASIF_NO_NORMAL'] == 1][['CLASIF_SBS']]

In [ ]:
base['DES_GRUPO_ECONOMICO'].value_counts(dropna=False)

In [ ]:
base.shape

In [ ]:
base['NO_EST_ASOCIADO_A_GRUPO_ECONOMICO'] = base['DES_GRUPO_ECONOMICO'].apply(lambda _: pd.isnull(_) or _.strip() == '').astype(int)
base['NO_EST_ASOCIADO_A_GRUPO_ECONOMICO'].value_counts()

## ANALISIS DE GRUPO ECONOMICO

In [ ]:
#try:
#    connection_b20.close()
#except:
#    pass

try:
    connection_market.close()
except:
    pass


#conneciom MARKET
connection_market = pymssql.connect(host='10.11.12.90\BDT',
                             user='ibetlmarket',
                             port=1433,
                             password='m@rk3t2o15',
                             database='MARKET'
                            )

In [ ]:
ge = """
    SELECT * FROM ODS.HD_VMVP_VPC_GRUPO_ECONOMICO_ALERTA with (nolock)
"""
ge = pd.read_sql(
   ge , connection_market
)
ge.head(3)

In [ ]:
ge.shape, ge['PERIODO'].max()

In [ ]:
ge =  ge[(ge['DES_GRUPO_ECONOMICO'].notnull()) & (ge['DES_GRUPO_ECONOMICO'] != '')]
ge.shape

In [ ]:
#list(range(9))

In [ ]:
ge['PERCENT_GRUPO_ECON_VPC_SEGUIR'] = ge['NRO_FEVE_SEGUIR'] / ge['NRO_MIEMBROS_GRUPO_ECONOMICO']
ge['PERCENT_GRUPO_ECON_VPC_FEVE_DURO'] = ge['NRO_FEVE_DURO'] / ge['NRO_MIEMBROS_GRUPO_ECONOMICO']
ge['PERCENT_GRUPO_ECON_VPC_NO_CLASIF_NORMAL'] = ge['NRO_CLASIF_NO_NORMAL'] / ge['NRO_MIEMBROS_GRUPO_ECONOMICO']

for orden in range(6):
    print(orden)
    ge['TIENE_{}_O_MAS_GRUPO_ECON_VPC_SEGUIR'.format(orden +1)] =  ge['NRO_FEVE_SEGUIR'].apply(lambda _: _ >= orden + 1).astype(int)
    ge['TIENE_{}_O_MAS_GRUPO_ECON_VPC_FEVE_DRO'.format(orden +1)] = ge['NRO_FEVE_DURO'].apply(lambda _: _ >= orden + 1).astype(int)
    ge['TIENE_{}_O_MAS_GRUPO_ECON_VPC_NO_CLASIF_NORMAL'.format(orden +1)] = ge['NRO_CLASIF_NO_NORMAL'].apply(lambda _: _ >= orden + 1).astype(int)


for orden in range(10, 40, 10):
    ge['TIENE_{}PERCENT_O_MAS_GRUPO_ECON_VPC_SEGUIR'.format(orden)] =  ge['NRO_FEVE_SEGUIR'].apply(lambda _: _>= orden/100).astype(int)
    ge['TIENE_{}PERCENT_O_MAS_GRUPO_ECON_VPC_FEVE_DRO'.format(orden)] = ge['NRO_FEVE_DURO'].apply(lambda _: _>= orden/100).astype(int)
    ge['TIENE_{}PERCENT_O_MAS_GRUPO_ECON_NO_CLASIF_NORMAL'.format(orden)] = ge['NRO_CLASIF_NO_NORMAL'].apply(lambda _: _>= orden/100).astype(int)
    
    

ge = ge.rename(columns={
    'NRO_CLASIF_NO_NORMAL': 'NRO_CLASIF_NO_NORMAL_GRUPO_EECON',
    'NRO_FEVE_SEGUIR': 'NRO_FEVE_SEGUIR_GRUPO_EECON',
    'NRO_FEVE_DURO': 'NRO_FEVE_DURO_GRUPO_EECON'
})

ge.head()

In [ ]:
base.shape

In [ ]:
ge['PERIODO'] = ge['PERIODO'].astype(str)
base = base.merge(
    ge, how='left', on=['PERIODO', 'DES_GRUPO_ECONOMICO']
)
base.shape

#### >>> HISTORICOS DE COMPORTAMIENTO CLIENTE EN GRUPO ECONÓMICO

In [ ]:
for col_percent in ['PERCENT_GRUPO_ECON_VPC_SEGUIR', 'PERCENT_GRUPO_ECON_VPC_FEVE_DURO', 'PERCENT_GRUPO_ECON_VPC_NO_CLASIF_NORMAL']:
    
    name = col_percent.split('VPC_')[-1]
    print(">"*50, name)
    acum = generate_listado_u3m(
        ge, 'listados_percent', values=col_percent,
        index=['DES_GRUPO_ECONOMICO'], columns=['PERIODO'], aggfunc=np.max
    )
    acum['TENDENCIA_PERCENT_{}_GRUPO_ECON_U7M'.format(name)] = acum['listados_percent'].apply(lambda _: np.polyfit(list(range(1, 8)), _[-7:], 1)[0])
    acum['GRUPO_ECON_TIENE_TENDENCIA_AUMENTO_{}_U7M'.format(name)] = acum['TENDENCIA_PERCENT_{}_GRUPO_ECON_U7M'.format(name)].apply(lambda _: _ > 0).astype(int)
    
    acum['TENDENCIA_PERCENT_{}_GRUPO_ECON_U3M'.format(name)] = acum['listados_percent'].apply(lambda _: np.polyfit(list(range(1, 4)), _[-3:], 1)[0])
    acum['GRUPO_ECON_TIENE_TENDENCIA_AUMENTO_{}_U3M'.format(name)] = acum['TENDENCIA_PERCENT_{}_GRUPO_ECON_U3M'.format(name)].apply(lambda _: _ > 0).astype(int)
    

In [ ]:
acum.head()

In [ ]:
acum['GRUPO_ECON_MANTIENE_TENDENCIA_CLASIF_NO_NORMAL'] = acum[['TENDENCIA_PERCENT_NO_CLASIF_NORMAL_GRUPO_ECON_U3M', 'TENDENCIA_PERCENT_NO_CLASIF_NORMAL_GRUPO_ECON_U7M']].apply(
    lambda _: _[0] > 0 and _[1] > 0, axis=1
).astype(int)

acum['GRUPO_ECON_MANTIENE_TENDENCIA_FUERTE_CLASIF_NO_NORMAL'] = acum[['TENDENCIA_PERCENT_NO_CLASIF_NORMAL_GRUPO_ECON_U3M', 'TENDENCIA_PERCENT_NO_CLASIF_NORMAL_GRUPO_ECON_U7M']].apply(
    lambda _: _[0] > 0.001 and _[1] > 0.001, axis=1
).astype(int)

pd.crosstab(
    acum['GRUPO_ECON_MANTIENE_TENDENCIA_CLASIF_NO_NORMAL'], acum['PERIODO']
)

In [ ]:
acum['PERIODO'] = acum['PERIODO'].astype(str)
base = base.merge(
    acum, how='left', on=['PERIODO', 'DES_GRUPO_ECONOMICO']
)
base.shape

## ANALISIS SEI MERCADO

In [ ]:
ge = """
    SELECT * FROM ODS.HD_VMVP_VPC_SEI_ALERTA_CARTERA WITH (NOLOCK)
"""
ge = pd.read_sql(
   ge , connection_market
)
ge['PERCENT_SEI_VPC_SEGUIR'] = ge['NRO_FEVE_SEGUIR'] / ge['NRO_MISMO_SEI']
ge['PERCENT_SEI_VPC_FEVE_DURO'] = ge['NRO_FEVE_DURO'] / ge['NRO_MISMO_SEI']
ge['PERCENT_SEI_VPC_NO_CLASIF_NORMAL'] = ge['NRO_CLASIF_NO_NORMAL'] / ge['NRO_MISMO_SEI']

ge.head()

In [ ]:
print(ge.shape)
ge =  ge[(ge['SEI_MCDO'].notnull()) & (ge['SEI_MCDO'] != '')]
ge.shape

In [ ]:
ge['PERCENT_SEI_VPC_NO_CLASIF_NORMAL'].hist()

In [ ]:
ge['PERCENT_SEI_VPC_SEGUIR'].hist()

### >> TOP PERCENTILES DE LA ÚLTIMA COSECHA

In [ ]:
top_percent_seguir = ge['PERCENT_SEI_VPC_SEGUIR'].quantile(0.75)
top_percent_seguir

In [ ]:
ge[ge['PERCENT_SEI_VPC_SEGUIR'] >= top_percent_seguir]['SEI_MCDO'].unique()

In [ ]:
ge['SEI_TOP_PERCENT_FEVE_SEGUIR'] = ge['PERCENT_SEI_VPC_SEGUIR'].apply(lambda _: _ >= top_percent_seguir).astype(int)

In [ ]:
#NRO_CLASIF_NO_NORMAL 	NRO_FEVE_DURO 	NRO_FEVE_SEGUIR

In [ ]:
for orden in range(10, 50, 5):
    print(orden, end=' ')
    ge['TIENE_{}PERCENT_O_MAS_SEI_VPC_SEGUIR'.format(orden)] =  ge['NRO_FEVE_SEGUIR'].apply(lambda _: _>= orden/100).astype(int)
    ge['TIENE_{}PERCENT_O_MAS_SEI_VPC_FEVE_DRO'.format(orden)] = ge['NRO_FEVE_DURO'].apply(lambda _: _>= orden/100).astype(int)
    ge['TIENE_{}PERCENT_O_MAS_SEI_NO_CLASIF_NORMAL'.format(orden)] = ge['NRO_CLASIF_NO_NORMAL'].apply(lambda _: _>= orden/100).astype(int)


ge = ge.rename(columns={
    'NRO_CLASIF_NO_NORMAL': 'NRO_CLASIF_NO_NORMAL_SEI',
    'NRO_FEVE_SEGUIR': 'NRO_FEVE_SEGUIR_SEI',
    'NRO_FEVE_DURO': 'NRO_FEVE_DURO_SEI'
})
ge.head()

In [ ]:
ge['PERIODO'] = ge['PERIODO'].astype(str)
base = base.merge(
    ge, how='left', on=['PERIODO', 'SEI_MCDO']
)
base.shape

In [ ]:
for col_percent in ['PERCENT_SEI_VPC_SEGUIR', 'PERCENT_SEI_VPC_FEVE_DURO', 'PERCENT_SEI_VPC_NO_CLASIF_NORMAL']:
    
    name = col_percent.split('VPC_')[-1]
    print(">"*50, name)
    acum = generate_listado_u3m(
        ge, 'listados_percent', values=col_percent,
        index=['SEI_MCDO'], columns=['PERIODO'], aggfunc=np.max
    )
    acum['TENDENCIA_PERCENT_{}_SEI_U7M'.format(name)] = acum['listados_percent'].apply(lambda _: np.polyfit(list(range(1, 8)), _[-7:], 1)[0])
    acum['SEI_TIENE_TENDENCIA_AUMENTO_{}_U7M'.format(name)] = acum['TENDENCIA_PERCENT_{}_SEI_U7M'.format(name)].apply(lambda _: _ > 0).astype(int)
    
    acum['TENDENCIA_PERCENT_{}_SEI_U3M'.format(name)] = acum['listados_percent'].apply(lambda _: np.polyfit(list(range(1, 4)), _[-3:], 1)[0])
    acum['SEI_TIENE_TENDENCIA_AUMENTO_{}_U3M'.format(name)] = acum['TENDENCIA_PERCENT_{}_SEI_U3M'.format(name)].apply(lambda _: _ > 0).astype(int)

In [ ]:
acum.head()

In [ ]:
acum['SEI_MANTIENE_TENDENCIA_CRECIENTE_FUERTE_CLASIF_NO_NORMAL'] = acum[['TENDENCIA_PERCENT_NO_CLASIF_NORMAL_SEI_U3M', 'TENDENCIA_PERCENT_NO_CLASIF_NORMAL_SEI_U7M']].apply(
    lambda _: _[0] > 0.001 and _[1] > 0.001, axis=1
).astype(int)

acum['SEI_MANTIENE_TENDENCIA_CRECIENTE_CLASIF_NO_NORMAL'] = acum[['TENDENCIA_PERCENT_NO_CLASIF_NORMAL_SEI_U3M', 'TENDENCIA_PERCENT_NO_CLASIF_NORMAL_SEI_U7M']].apply(
    lambda _: _[0] > 0 and _[1] > 0, axis=1
).astype(int)

pd.crosstab(
    acum['SEI_MANTIENE_TENDENCIA_CRECIENTE_CLASIF_NO_NORMAL'], acum['PERIODO']
)

In [ ]:
acum['PERIODO'] = acum['PERIODO'].astype(str)
base = base.merge(
    acum, how='left', on=['PERIODO', 'SEI_MCDO']
)
base.shape

In [ ]:
#base = new_dataFrame.merge(
#    base[['COD_UNICO', 'BUCKET', 'FEVE_FICHA', 'ESTADO_FICHA', 'RIESGO', 'TICKET_PROMEDIO_EN_RIESGO', 'UNIVERSO_MESA']],
#    how='left', on=['COD_UNICO']
#)
#del new_dataFrame
base['PERIODO'].value_counts()

In [ ]:
gc.collect()
del acum
del ge

In [ ]:
base.shape

In [ ]:
base['CLASIF_NO_NORMAL'] = base['CLASIF_NO_NORMAL'].astype(int)

In [ ]:
pd.crosstab(
    base['CLASIF_NO_NORMAL'], base['PERIODO']
)

In [ ]:
#base['PERIODO'].value_counts(dropna=False).sort_index(ascending=False)

### LINEAS

In [ ]:
try:
    connection_market.close()
except:
    pass

#conneciom MARKET
connection_market = pymssql.connect(**dicc_conection_market)
gc.collect()

In [ ]:
sql0 = """
    SELECT *
    FROM ODS.HD_VMVP_LINEAS_ALERTAS_CARTERA_BE  WITH (NOLOCK)
"""

lineas = pd.read_sql(
   sql0 , connection_market
)
lineas['PERIODO'] = lineas['PERIODO'].astype(str)
lineas['COD_UNICO'] = lineas['COD_UNICO'].astype(str)
lineas.head(2)

In [ ]:
lineas['PERIODO'].max(), lineas['PERIODO'].min(), lineas.shape, lineas.groupby(by=['COD_UNICO', 'PERIODO']).size().shape

In [ ]:
lineas['PERIODO'].value_counts(dropna=False)

In [ ]:
lineas['ESTADO_LINEA'].value_counts(dropna=False)

In [ ]:
lineas['ESTADO_LINEA'] = lineas['ESTADO_LINEA'].apply(lambda _: _.upper())
lineas['ESTADO_LINEA'].value_counts(dropna=False)

#### Buscando viegentes

In [ ]:
acum = generate_listado_u3m(
    lineas[lineas['ESTADO_LINEA'] == 'VIGENTE'], 'listados_vigentes_lineas', values='MAX_VENCIMIENTO',
    index=['COD_UNICO'], columns=['PERIODO'], aggfunc=len
)
acum.head()

In [ ]:
def searhc_no_zero(lista):
    meses = 0
    for val in lista[::-1]:
        if val > 0:
            return meses
        meses += 1
    
    return meses

acum['NO_TIENE_LINEA_VIGENTE_MES_ACTUAL'] = acum['listados_vigentes_lineas'].apply(lambda _: _[-1] == 0).astype(int)
acum['TIENE_LINEAS_VIGENTES_MES_ACTUAL'] = acum['listados_vigentes_lineas'].apply(lambda _: _[-1] == 1).astype(int)
acum['NRO_MESES_SIN_LINEA_VIGENTE'] = acum['listados_vigentes_lineas'].apply(searhc_no_zero).astype(int)
acum['TIENE_UNA_O_MAS_LINEAS_VIGENTES_U2M'] = acum['listados_vigentes_lineas'].apply(lambda _: _[-1] > 0 or _[-2] > 0).astype(int)
acum['TIENE_UNA_O_MAS_LINEAS_VIGENTES_U3M'] = acum['listados_vigentes_lineas'].apply(lambda _: _[-1] > 0 or _[-2] > 0 or _[-3] > 0).astype(int)

del acum['listados_vigentes_lineas']

In [ ]:
print(base.shape)

acum['PERIODO'] = acum['PERIODO'].astype(str)
base = base.merge(
    acum, how='left', on=['PERIODO', 'COD_UNICO']
)
print(base.shape)

### COMPARACION FLAG_VIGENTE WORK FLOW vs EQUIPO LINEAS

In [ ]:
for periodo in sorted(base['PERIODO'].unique()):
    print(">> ", periodo)
    
    display(pd.crosstab(
        base[base['PERIODO'] == periodo]['FLG_VIGENTE'].fillna(-1), base[base['PERIODO'] == periodo]['TIENE_LINEAS_VIGENTES_MES_ACTUAL'].fillna(-1)
    ))
    
    print("/"*100)

In [ ]:
base['FEC_INGRESO'] = pd.to_datetime(
    base['FEC_INGRESO'], errors='coerce', format='%Y-%m-%d'
)
base['FEC_PRIMER_PRODUCTO_APERTURADO'] = pd.to_datetime(
    base['FEC_PRIMER_PRODUCTO_APERTURADO'], errors='coerce', format='%Y-%m-%d'
)

def apply_diff_date(serie_diff):
    return (serie_diff / np.timedelta64(1,'M')).round(3)

base['DIFF_TIME_INICIO_IBK_FIRST_PRODUCT'] = apply_diff_date(
    base['FEC_PRIMER_PRODUCTO_APERTURADO'] - base['FEC_INGRESO']
)

In [ ]:
base['DIFF_TIME_INICIO_IBK_FIRST_PRODUCT'].hist()

In [ ]:
base['PERIODO_DATE'] = pd.to_datetime(
    base['PERIODO'], errors='coerce', format='%Y%m'
)
base['ANTIGUEDAD_IBK'] = apply_diff_date(
    base['PERIODO_DATE'] - base['FEC_INGRESO']
)
base['ANTIGUEDAD_IBK'].hist()

#### >>> WORK FLOW dice SI, equipo lineas dice NO

In [ ]:
base[
    (base['PERIODO'] == '202103') & (base['TIENE_LINEAS_VIGENTES_MES_ACTUAL'] == 0) & (base['FLG_VIGENTE'] == 1)
][['COD_UNICO', 'NOMBRE_COMPLETO', 'FEC_INGRESO']].head(10)

#### >>> WORK FLOW dice SI, equipo lineas dice no tiene vigente despues del 2019

In [ ]:
base[
    (base['PERIODO'] == '202103') & (base['TIENE_LINEAS_VIGENTES_MES_ACTUAL'].isnull()) & (base['FLG_VIGENTE'] == 1)
][['COD_UNICO', 'NOMBRE_COMPLETO', 'FEC_INGRESO']].head(10)

#### >>> WORK FLOW dice NO, equipo lineas dice SI

In [ ]:
base[
    (base['PERIODO'] == periodo) & (base['TIENE_LINEAS_VIGENTES_MES_ACTUAL'] == 1) & (base['FLG_VIGENTE'] == 0)
][['COD_UNICO', 'NOMBRE_COMPLETO']].head(10)

In [ ]:
base['TIENE_2MESES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _: _[0] == 2 and _[1] >= 2, axis=1).astype(int)
base['TIENE_3MESES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _:  _[0] == 3 and _[1] >= 3, axis=1).astype(int)
base['TIENE_4MESES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _:  _[0] == 4 and _[1] >= 4, axis=1).astype(int)

base['TIENE_MAS_DE_1MES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _:  _[0] > 1 and _[1] > 1, axis=1).astype(int)
base['TIENE_MAS_DE_2MESES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _: _[0] > 2 and _[1] > 2, axis=1).astype(int)
base['TIENE_MAS_DE_3MESES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _: _[0] > 3 and _[1] > 3, axis=1).astype(int)
base['TIENE_MAS_DE_4MESES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _: _[0] > 4 and _[1] > 4, axis=1).astype(int)
base['TIENE_MAS_DE_5MESES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _: _[0] > 5 and _[1] > 5, axis=1).astype(int)
base['TIENE_MAS_DE_6MESES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _: _[0] > 6 and _[1] > 6, axis=1).astype(int)
base['TIENE_MAS_DE_7MESES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _: _[0] > 7 and _[1] > 7, axis=1).astype(int)
base['TIENE_MAS_DE_8MESES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _: _[0] > 8 and _[1] > 8, axis=1).astype(int)
base['TIENE_MAS_DE_9MESES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _: _[0] > 9 and _[1] > 9, axis=1).astype(int)
base['TIENE_MAS_DE_10MESES_SIN_LINEA_VIGENTE']  = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'ANTIGUEDAD_IBK']].apply(lambda _: _[0] > 10 and _[1] > 10, axis=1).astype(int)

In [ ]:
pd.crosstab(
    base['TIENE_MAS_DE_10MESES_SIN_LINEA_VIGENTE'].fillna(-1), base['PERIODO']
)

In [ ]:
base['NRO_MESES_SIN_LINEA_VIGENTE'].hist()

In [ ]:
base['NO_INCLUIDO_EN_BASE_LINEAS_CLIENTE'] = base['NRO_MESES_SIN_LINEA_VIGENTE'].apply(lambda _: 1 if pd.isnull(_) else 0)
base['NO_INCLUIDO_EN_BASE_LINEAS_CLIENTE'].value_counts()

In [ ]:
#base['PERIODO_DATE'] = pd.to_datetime(
#    base['PERIODO'], errors='coerce', format='%Y%m'
#)
#base['MIN_PERIODO_LINEAS_CLIENTE'] = lineas['PERIODO'].min()
#base['MIN_PERIODO_LINEAS_CLIENTE'] = pd.to_datetime(
#    base['MIN_PERIODO_LINEAS_CLIENTE'], errors='coerce', format='%Y%m'
#)

#def apply_diff_date(serie_diff):
#    return (serie_diff / np.timedelta64(1,'M')).round(3) 

#base['DIFF_WITH_MIN_PERIOD'] = apply_diff_date(
#    base['PERIODO_DATE'] - base['MIN_PERIODO_LINEAS_CLIENTE']
#)
#base['DIFF_WITH_MIN_PERIOD'].value_counts()

In [ ]:
#base['NRO_MESES_SIN_LINEA_VIGENTE'] = base[['NRO_MESES_SIN_LINEA_VIGENTE', 'DIFF_WITH_MIN_PERIOD']].apply(lambda _: _[1] if pd.isnull(_[0]) else _[0], axis=1)
#base['NRO_MESES_SIN_LINEA_VIGENTE'].hist()

In [ ]:
#for col in ['NO_TIENE_LINEA_VIGENTE_MES_ACTUAL', 'TIENE_2MESES_SIN_LINEA_VIGENTE', 'TIENE_3MESES_SIN_LINEA_VIGENTE', 'TIENE_3MESES_SIN_LINEA_VIGENTE',
#            'TIENE_MAS_DE_1MES_SIN_LINEA_VIGENTE', 'TIENE_MAS_DE_2MESES_SIN_LINEA_VIGENTE', 'TIENE_MAS_DE_3MESES_SIN_LINEA_VIGENTE',
#            'TIENE_MAS_DE_4MESES_SIN_LINEA_VIGENTE', 'TIENE_MAS_DE_5MESES_SIN_LINEA_VIGENTE', 'TIENE_MAS_DE_6MESES_SIN_LINEA_VIGENTE',
#            'TIENE_MAS_DE_9MESES_SIN_LINEA_VIGENTE', 'TIENE_MAS_DE_10MESES_SIN_LINEA_VIGENTE']:
#    base[col] = base[col].fillna(1)
#

#
#
#for col in ['TIENE_LINEAS_VIGENTES_MES_ACTUAL', 'TIENE_UNA_O_MAS_LINEAS_VIGENTES_U2M', 'TIENE_UNA_O_MAS_LINEAS_VIGENTES_U3M']:
#    base[col] = base[col].fillna(0)

### UTILIZACION

In [ ]:
utilizacion = lineas.groupby(by=['PERIODO', 'COD_UNICO'])['MONTOUTILIZADO'].max().reset_index()
utilizacion.head()

In [ ]:
utilizacion.tail()

In [ ]:
utilizacion['PERIODO'].max(), utilizacion['PERIODO'].min()

In [ ]:
print(base.shape)

acum['PERIODO'] = acum['PERIODO'].astype(str)
base = base.merge(
    utilizacion, how='left', on=['PERIODO', 'COD_UNICO']
)
print(base.shape)

In [ ]:
#lineas[lineas['COD_UNICO'] == '0000003392'].sort_values(by=['PERIODO'])[['PERIODO', 'MONTOUTILIZADO', 'ESTADO_LINEA']]

###  CONTEO DE LINEAS

In [ ]:
sql0 = """
    SELECT *
    FROM ODS.HD_VMVP_LINEAS_ORIGEN_ALERTAS_CARTERA_BE WITH (NOLOCK)
"""
lineas = pd.read_sql(
   sql0 , connection_market
)
lineas.head(2)

In [ ]:
acum = generate_listado_u3m(
    lineas[lineas['ESTADO_LINEA'] == 'VIGENTE'], 'listados_vigentes', values='NRO_LINEA',
    index=['COD_UNICO'], columns=['PERIODO'], aggfunc=len
)
acum.head()

In [ ]:
acum.tail()

In [ ]:
acum['TIENE_MENOR_NRO_LINEAS_VIGENTES_U3M'] = acum['listados_vigentes'].apply(lambda _: _[-3] >= _[-2] and _[-2 ] > _[-1] ).astype(int)
acum['TIENE_MAYOR_NRO_LINEAS_VIGENTES_U3M'] = acum['listados_vigentes'].apply(lambda _: _[-3] <= _[-2] and _[-2 ] < _[-1] ).astype(int)
acum['TIENE_DIMINUCION_1_LINEA_VIGENTE_MES_ANT'] = acum['listados_vigentes'].apply(lambda _: _[-2 ] - _[-1] == 1).astype(int)
acum['TIENE_DIMINUCION_2_LINEAS_VIGENTES_MES_ANT'] = acum['listados_vigentes'].apply(lambda _: _[-2 ] - _[-1] == 2).astype(int)
acum['TIENE_1_LINEA_VIGENTE_U3M'] = acum['listados_vigentes'].apply(lambda _: sum(_[-3:]) / 3 <= 1 and sum(_[-3:]) / 3 > 0).astype(int)
acum['TIENE_2_LINEA_VIGENTE_U3M'] = acum['listados_vigentes'].apply(lambda _: sum(_[-3:]) / 3 <= 2 and sum(_[-3:]) / 3 > 1).astype(int)
acum['TIENE_MAS_DE_2_LINEAS_VIGENTES_U3M'] = acum['listados_vigentes'].apply(lambda _: sum(_[-3:]) / 3 > 2).astype(int)
acum['TIENE_MAS_DE_3_LINEAS_VIGENTES_U3M'] = acum['listados_vigentes'].apply(lambda _: sum(_[-3:]) / 3 > 3).astype(int)
acum['TIENE_MAS_DE_4_LINEAS_VIGENTES_U3M'] = acum['listados_vigentes'].apply(lambda _: sum(_[-3:]) / 3 > 4).astype(int)
del acum['listados_vigentes']
acum.tail()

In [ ]:
acum['PERIODO'] = acum['PERIODO'].astype(str)
base = base.merge(
    acum, how='left', on=['PERIODO', 'COD_UNICO']
)
base.shape

## NRO DE TRBAJADORES

In [ ]:
#try:
#    connection_b20.close()
#except:
#    pass

try:
    connection_market.close()
except:
    pass

#conneciom MARKET
connection_market = pymssql.connect(**dicc_conection_market)

In [ ]:
sql_ = """
    SELECT *
    FROM ODS.HM_VMVP_NRO_TRABAJADOES_ALERTA_CARTERA WITH (NOLOCK)
"""
workers = pd.read_sql(
   sql_ , connection_market
).drop(['FECHA_CARGA'], axis=1)
workers.head(2)

In [ ]:
workers = workers.groupby(by=['RUC', 'PERIODO']).agg(
    {'NRO_TRABAJADORES': sum, 'NRO_PENSIONISTAS': sum, 'NRO_PRESTADORES_SERVICIO': sum}
).reset_index()

workers.head()

In [ ]:
workers['PERIODO'].max()

In [ ]:
print(workers.shape)

In [ ]:
# Se auemnta un periodo por el desfase de la información
workers['PERIODO'] = workers['PERIODO'] .apply(
    lambda _: (datetime.strptime(_, '%Y%m') + relativedelta(months=1)).strftime('%Y%m')
)

In [ ]:
workers[workers['RUC'] == '20100340438']

In [ ]:
workers['PERIODO'].value_counts().sort_index()

## PROVISIONAL

In [ ]:
_202102 = workers[workers['PERIODO'] == '202101']
_202102['PERIODO'] = '202102'

_202103 = workers[workers['PERIODO'] == '202101']
_202103['PERIODO'] = '202103'

_202104 = workers[workers['PERIODO'] == '202101']
_202104['PERIODO'] = '202104'

_202105 = workers[workers['PERIODO'] == '202101']
_202105['PERIODO'] = '202105'

workers = pd.concat([workers, _202102, _202103, _202104, _202105], axis=0, ignore_index=True)
workers['PERIODO'].value_counts().sort_index()

In [ ]:
workers.dtypes

In [ ]:
workers[workers['RUC'] == '20100340438']

In [ ]:
#workers.dtypes

In [ ]:
workers.isnull().sum()

In [ ]:
workers['PERIODO'] = workers['PERIODO'].astype(int)

In [ ]:
pivoteo = pd.pivot_table(
    workers[['RUC', 'PERIODO', 'NRO_TRABAJADORES']], 
    index=['RUC'], columns=['PERIODO'], aggfunc=np.max, values='NRO_TRABAJADORES', fill_value=-1
)

columnas_pivoteo = sorted([int(_) for _ in pivoteo.columns])
print(columnas_pivoteo)

pivoteo[columnas_pivoteo].head(20)

In [ ]:
pivoteo = pivoteo[columnas_pivoteo].reset_index()
#del pivoteo['PERIODO']
pivoteo.columns = [str(col) for col in pivoteo.columns]
pivoteo.head(1)

In [ ]:
def reemplza_con_previo(lista):
    if lista[0] == -1:
        return lista[1]
    
    return lista[0]

for ii, col in enumerate(columnas_pivoteo[1:], start=1):
    
    columna_previa = str(columnas_pivoteo[ii-1])
    columna_actual = str(col)
    print(ii, columna_actual, columna_previa)
    
    pivoteo[columna_actual] = pivoteo[[columna_actual, columna_previa]].apply(
        lambda _: reemplza_con_previo(_),
        axis=1
    )
    
pivoteo.head(20)

In [ ]:
pivoteo.columns

In [ ]:
#pivoteo['list_'] = pivoteo[[col for col in pivoteo.columns if col not in ['RUC', 'list_']]].apply(list, axis=1)
#pivoteo.head()

In [ ]:
#for col in [_ for _ in pivoteo.columns if _ not in ['RUC', 'list_']]:
#    del pivoteo[col]

In [ ]:
#pivoteo['list_'] = pivoteo['list_'].apply(lambda _: [val for val in _ if val != -1])
#pivoteo['len_'] = pivoteo['list_'].apply(len)
#pivoteo.head()

In [ ]:
workers = pd.melt(pivoteo, id_vars=['RUC'], var_name='PERIODO', value_name='NRO_TRABAJADORES')
workers.head(3)

In [ ]:
print(workers.shape)
workers = workers[workers['NRO_TRABAJADORES'] != -1]
print(workers.shape)

In [ ]:
workers[workers['RUC'] == '20100340438']

In [ ]:
workers[workers['RUC'] == '20100227542']

In [ ]:
acum = generate_listado_u3m(
    workers, 'listados_workers', values='NRO_TRABAJADORES', index=['RUC'], columns=['PERIODO'], aggfunc=sum
)
acum.head()

In [ ]:
workers.tail()

In [ ]:
workers.shape, workers.groupby(['PERIODO', 'RUC']).size().shape

In [ ]:
workers.dtypes

In [ ]:
print(base.shape)
base = base.merge(
    workers, how='left', on=['PERIODO', 'RUC']
)
base.shape

In [ ]:
acum['listados_workers'].head(3)

### >> CREACION D EVARIABLES DE PERSONAL RELEVANTES

In [ ]:
def diff_hace_10_meses(lista):
    # RELEVANTES
    try:
        return lista[-10] - lista[-1] > 5
    except:
        return lista[0] - lista[-1] > 5
    
acum['TIENE_DISMINUCION_PERSONAL_COMP_10MATRAS'] = acum['listados_workers'].apply(lambda _: diff_hace_10_meses(_) ).astype(int)
acum['TENDENCIA_COMPORTAMIENTO_NRO_TRABAJADORES_SUNAT'] = acum['listados_workers'].apply(lambda _: np.polyfit(list(range(1, 8)), _[-7:], 1)[0])
acum['TENDENCIA_A_DISMINUIR_NRO_TRABAJADORES_SUNAT_U7M'] = acum['TENDENCIA_COMPORTAMIENTO_NRO_TRABAJADORES_SUNAT'].apply(lambda _: _ < 0).astype(int)

In [ ]:
acum['NRO_TRABAJADORESX_U2M'] = acum['listados_workers'].apply(lambda _: _[-1])
acum['NRO_TRABAJADORESX_U7M'] = acum['listados_workers'].apply(lambda _: _[-6])
acum['NRO_TRABAJADORESX_U8M'] = acum['listados_workers'].apply(lambda _: _[-7])
cols_diminucon = ['NRO_TRABAJADORESX_U7M', 'NRO_TRABAJADORESX_U8M', 'NRO_TRABAJADORESX_U2M']

In [ ]:
def get_percent_(hace_7, hace_8, hace_2, treshold):
    media_hace_7m =(hace_8 + hace_7) / 2
    #print("media_hace_7m : ", media_hace_7m)
    if media_hace_7m == 0:
        return 0
    
    # RELEVANTES
    if media_hace_7m - hace_2 < 5: ## NO ES RELEVANTE
        return 0
    
    return hace_2/media_hace_7m <= treshold

acum['TIENE_DISMINUCION_PERSONAL_1PERCENT_A_MAS_U7M'] = acum[cols_diminucon].apply(lambda _: get_percent_(_[0],_[1],_[2],0.99), axis=1).astype(int)
acum['TIENE_DISMINUCION_PERSONAL_3PERCENT_A_MAS_U7M'] = acum[cols_diminucon].apply(lambda _: get_percent_(_[0],_[1],_[2], 0.97), axis=1).astype(int)
acum['TIENE_DISMINUCION_PERSONAL_5PERCENT_A_MAS_U7M'] = acum[cols_diminucon].apply(lambda _: get_percent_(_[0],_[1],_[2], 0.95), axis=1).astype(int)
acum['TIENE_DISMINUCION_PERSONAL_10PERCENT_A_MAS_U7M'] = acum[cols_diminucon].apply(lambda _: get_percent_(_[0],_[1],_[2], 0.90), axis=1).astype(int)
acum['TIENE_DISMINUCION_PERSONAL_20PERCENT_A_MAS_U7M'] = acum[cols_diminucon].apply(lambda _: get_percent_(_[0],_[1],_[2], 0.8), axis=1).astype(int)
acum['TIENE_DISMINUCION_PERSONAL_30PERCENT_A_MAS_U7M'] = acum[cols_diminucon].apply(lambda _: get_percent_(_[0],_[1],_[2], 0.7), axis=1).astype(int)
acum['TIENE_DISMINUCION_PERSONAL_40PERCENT_A_MAS_U7M'] = acum[cols_diminucon].apply(lambda _: get_percent_(_[0],_[1],_[2], 0.6), axis=1).astype(int)
acum['TIENE_DISMINUCION_PERSONAL_50PERCENT_A_MAS_U7M'] = acum[cols_diminucon].apply(lambda _: get_percent_(_[0],_[1],_[2], 0.5), axis=1).astype(int)
acum['TIENE_DISMINUCION_PERSONAL_60PERCENT_A_MAS_U7M'] = acum[cols_diminucon].apply(lambda _: get_percent_(_[0],_[1],_[2], 0.4), axis=1).astype(int)
acum['NO_TIENE_TRABAJADORES_REGISTRADOS'] = acum['listados_workers'].apply(lambda _: sum(_) == 0).astype(int)
acum['NUMERO_TRABAJADORES_REGISTRADOS_U1M'] = acum['listados_workers'].apply(lambda _: _[-1])
acum['NUMERO_TRABAJADORES_REGISTRADOS_U3M'] = acum['listados_workers'].apply(lambda _: _[-2])
acum['NUMERO_TRABAJADORES_REGISTRADOS_U6M'] = acum['listados_workers'].apply(lambda _: _[-5])
acum['NUMERO_TRABAJADORES_REGISTRADOS_U10M'] = acum['listados_workers'].apply(lambda _: _[0])
del acum['listados_workers']

acum.head()

In [ ]:
acum[
    (acum['RUC'] == '10028918041') & (acum['PERIODO'] == 202101)
]

In [ ]:
acum['RUC'] = acum['RUC'].astype(str)
acum['PERIODO'] = acum['PERIODO'].astype(str)

base = base.merge(
    acum, how='left', on=['PERIODO', 'RUC']
)
base.shape

# REACTIVA

In [ ]:
try:
    connection_market.close()
except:
    pass


#conneciom MARKET
connection_market = pymssql.connect(host='10.11.12.90\BDT',
                             user='ibetlmarket',
                             port=1433,
                             password='m@rk3t2o15',
                             database='MARKET'
                            )

In [ ]:
sql1 = """
    SELECT * FROM ODS.HD_VMVP_REACTIVA_IBK_ALERTAS_CARTERA_BE WITH (NOLOCK)
"""
interno_reactiva_ibk = pd.read_sql(
   sql1 , connection_market
)
interno_reactiva_ibk.head(2)

In [ ]:
interno_reactiva_ibk['PERIODO'] = interno_reactiva_ibk['FECHA_DESEMBOLSO'].apply(lambda _: int(str(_).replace('-', '')[:6]))
interno_reactiva_ibk['PERIODO'].value_counts(dropna=False)

In [ ]:
pivoteo = pd.pivot_table(
    interno_reactiva_ibk[['CODUNICOCLI', 'PERIODO', 'TIPO_REACTIVA']], 
    index=['CODUNICOCLI'], columns=['PERIODO'], aggfunc=len, values='TIPO_REACTIVA',fill_value=0
)
pivoteo.head()

In [ ]:
# Adaptacion Rractiva faltantes en meses psoteriores a Noviembre --- 07/01/2021

columnas_pivoteo = [int(_) for _ in pivoteo.columns]                 
for column in [mes for mes in rango_reactiva_int if int(mes) >= columnas_pivoteo[0]]:
    if column not in columnas_pivoteo:
        pivoteo[column] = 0

pivoteo.head()

In [ ]:
columnas_ordenadas = sorted(pivoteo.columns, reverse=False)

for col in columnas_ordenadas[1:]:
    posicion_columna = columnas_ordenadas.index(col)
    #print(posicion_columna)
    print(posicion_columna, col, columnas_ordenadas[posicion_columna -1])
    pivoteo[col] = pivoteo[
        [columnas_ordenadas[posicion_columna -1], col]
    ].sum(axis=1)

pivoteo.head()

In [ ]:
pivoteo = pivoteo.unstack().reset_index()
pivoteo.columns = ['PERIODO', 'COD_UNICO', 'NRO_VECES_REACTIVA_IBK']
pivoteo['TIENE_REACTIVA_IBK'] = pivoteo['NRO_VECES_REACTIVA_IBK'].apply(lambda _: _ > 0).astype(int)
pivoteo['TIENE_REACTIVA1_REACTIVA2_IBK'] = pivoteo['NRO_VECES_REACTIVA_IBK'].apply(lambda _: _ > 1).astype(int)

display(pivoteo['TIENE_REACTIVA1_REACTIVA2_IBK'].value_counts(dropna=False))
pivoteo.head(3)

In [ ]:
pivoteo['PERIODO'] = pivoteo['PERIODO'].astype(str)
base = base.merge(
    pivoteo, how='left', on=['PERIODO', 'COD_UNICO']
)
base.shape

### REACTIVA RCC

In [ ]:
try:
    connection_market.close()
except:
    pass


#conneciom MARKET
connection_market = pymssql.connect(host='10.11.12.90\BDT',
                             user='ibetlmarket',
                             port=1433,
                             password='m@rk3t2o15',
                             database='MARKET'
                            )

In [ ]:
sql_reactiva_rcc = """
    SELECT * FROM ODS.HD_VMVP_REACTIVA_ALERTAS_CARTERA_BE WITH (NOLOCK)
"""
reactiva = pd.read_sql(
   sql_reactiva_rcc , connection_market  ## # aCTUALIZACION TABLA 
)

reactiva['PERIODO'].max(), reactiva['PERIODO'].min()

In [ ]:
## Se le suma un mes para que la info de rcc se use el mes siguiente
reactiva['PERIODO'] = reactiva['PERIODO'] .apply(
    lambda _: (datetime.strptime(_, '%Y%m') + relativedelta(months=1)).strftime('%Y%m')
)
pivoteo = pd.pivot_table(
    reactiva[['COD_UNICO', 'PERIODO', 'NBR_EMPRESA']], 
    index=['COD_UNICO'], columns=['PERIODO'], aggfunc=len, values='NBR_EMPRESA',fill_value=0
)

del reactiva
pivoteo.head()

In [ ]:
pivoteo = pivoteo.unstack().reset_index()
pivoteo.columns = ['PERIODO', 'COD_UNICO', 'NRO_VECES_REACTIVA_SF']
pivoteo['TIENE_REACTIVA_SF'] = pivoteo['NRO_VECES_REACTIVA_SF'].apply(lambda _: _ > 0).astype(int)
pivoteo['TIENE_REACTIVA1_REACTIVA2_SF'] = pivoteo['NRO_VECES_REACTIVA_SF'].apply(lambda _: _ > 1).astype(int)

display(pivoteo['TIENE_REACTIVA1_REACTIVA2_SF'].value_counts(dropna=False))
pivoteo.head(3)

In [ ]:
pd.crosstab(
    pivoteo['TIENE_REACTIVA_SF'].fillna(-1), pivoteo['PERIODO']
)

In [ ]:
pivoteo['PERIODO'] = pivoteo['PERIODO'].astype(str)
base = base.merge(
    pivoteo, how='left', on=['PERIODO', 'COD_UNICO']
)
base.shape

In [ ]:
base.head(2)

In [ ]:
for col in [_ for _ in base.columns if 'REACTIVA' in _.upper()]:
    base[col] = base[col].fillna(0)

base['TIENE_REACTIVA_CON_OTRA_EF'] = base[['TIENE_REACTIVA_SF', 'TIENE_REACTIVA_IBK']].apply(lambda _: 1 if _[0] == 1 and _[1] == 0 else 0, axis=1)
base['TIENE_REACTIVA_1_2_CON_OTRAS_EF'] = base[['TIENE_REACTIVA1_REACTIVA2_SF', 'TIENE_REACTIVA_IBK']].apply(
    lambda _: 1 if _[0] == 1 and _[1] == 0 else 0, axis=1)

base['TIENE_REACTIVA_SF']  = base[['TIENE_REACTIVA_SF', 'TIENE_REACTIVA_IBK']].apply(lambda _: 1 if _[0] == 1 or _[1] == 1 else 0, axis=1) 

base['TIENE_SOLO_1_CREDITO_REACTIVA_SF'] = base['NRO_VECES_REACTIVA_SF'].apply(lambda _: _ == 1).astype(int)
base['TIENE_REACTIVA_CON_OTRA_EF'].value_counts()

In [ ]:
base['TIENE_REACTIVA_1_2_CON_OTRAS_EF'].value_counts(dropna=False)

In [ ]:
base['TIENE_SOLO_1_CREDITO_REACTIVA_SF'].value_counts(dropna=False)

#### Review Cosechas JUlio y Agosto - REACTIVA IBK

In [ ]:
base[base['PERIODO'].astype(int) >= 202007]['TIENE_REACTIVA_IBK'].value_counts()

In [ ]:
base.isnull().sum()

In [ ]:
base.shape
base[base['PERIODO'].astype(int) >= 202007]['BUCKET'].value_counts()

In [ ]:
base.to_csv("AVANCE_1.csv", index=False)

In [ ]:
base['PERIODO'].value_counts().sort_index(ascending=False)

In [ ]:
clientes.shape

In [ ]:
pd.crosstab(
    base['TIENE_REACTIVA_SF'].fillna(-1), base['PERIODO']
)

In [ ]:
clientes.to_csv("clientes.csv", index=False)

In [ ]:
del clientes

In [ ]:
del base